# Liger kernel example with DDP, Mistral 7B instruct and openassistant-guanaco dataset

In the first course, we demonstrated a speed-up by using the *unsloth* library, which contains optimized GPU kernels created by manually deriving all compute heavy math steps. *Unsloth* only works for single GPU training though. Another library that also offers optimised GPU kernels is *liger*, which has the advantage that it can also be used for multi-GPU training.

In this notebook, we demonstrate finetuning of *Mistral 7B instruct* on two GPUs using DDP and *liger kernels*. The same script is run twice, once with without and once with *liger kernels* in order to compare speed and memory usage.

#### First, we write the python code to a file:

In [6]:
%%writefile phi3_guanaco_ddp_liger.py
# Import libraries
import torch
from accelerate import PartialState
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer, SFTConfig
import pynvml
import sys

if len(sys.argv) >= 2 and sys.argv[1] == '--enable-liger':
    enable_liger = True
    print('Using liger kernels.')
else:
    enable_liger = False
    print('Not using liger kernels.')

# Import liger kernels and apply automatic monkey-patching to models:
# from liger_kernel.transformers import apply_liger_kernel_to_phi3
# apply_liger_kernel_to_phi3()
# https://github.com/linkedin/Liger-Kernel?tab=readme-ov-file#getting-started
from liger_kernel.transformers import AutoLigerKernelForCausalLM

def print_gpu_utilization():
    pynvml.nvmlInit()
    device_count = pynvml.nvmlDeviceGetCount()
    memory_used = []
    for device_index in range(device_count):
        device_handle = pynvml.nvmlDeviceGetHandleByIndex(device_index)
        device_info = pynvml.nvmlDeviceGetMemoryInfo(device_handle)
        memory_used.append(device_info.used/1024**3)
    print('Memory occupied on GPUs: ' + ' + '.join([f'{mem:.1f}' for mem in memory_used]) + ' GB.')


# Choose a model and load tokenizer and model (using 4bit quantization):
model_name = '/leonardo_scratch/fast/EUHPC_D20_063/huggingface/models/mistralai--Mistral-7B-Instruct-v0.3'
# model_name = 'mistralai/Mistral-7B-Instruct-v0.3'
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token  # Needed for Mistral 7B

# For multi-GPU training, find out how many GPUs there are and which one we should use:
ps = PartialState()
num_processes = ps.num_processes
process_index = ps.process_index
local_process_index = ps.local_process_index

if enable_liger:
    model = AutoLigerKernelForCausalLM.from_pretrained(
        model_name,
        quantization_config=BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type='nf4',
            bnb_4bit_compute_dtype=torch.bfloat16,
        ),
        device_map={'':local_process_index},  # Changed for DDP
        attn_implementation='eager',  # 'eager', 'sdpa', or "flash_attention_2"
        trust_remote_code=True,
        torch_dtype=torch.bfloat16,
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type='nf4',
            bnb_4bit_compute_dtype=torch.bfloat16,
        ),
        device_map={'':local_process_index},  # Changed for DDP
        attn_implementation='eager',  # 'eager', 'sdpa', or "flash_attention_2"
        trust_remote_code=True,
        torch_dtype=torch.bfloat16,
    )


# Load the guanaco dataset
guanaco_train = load_dataset('/leonardo_scratch/fast/EUHPC_D20_063/huggingface/datasets/timdettmers--openassistant-guanaco', split='train')
guanaco_test = load_dataset('/leonardo_scratch/fast/EUHPC_D20_063/huggingface/datasets/timdettmers--openassistant-guanaco', split='test')
# guanaco_train = load_dataset('timdettmers/openassistant-guanaco', split='train')
# guanaco_test = load_dataset('timdettmers/openassistant-guanaco', split='test')
guanaco_train = guanaco_train.map(lambda entry: {
    'question1': entry['text'].split('###')[1].removeprefix(' Human: '),
    'answer1': entry['text'].split('###')[2].removeprefix(' Assistant: ')
})
guanaco_test = guanaco_test.map(lambda entry: {
    'question1': entry['text'].split('###')[1].removeprefix(' Human: '),
    'answer1': entry['text'].split('###')[2].removeprefix(' Assistant: ')
})
guanaco_train = guanaco_train.map(lambda entry: {'messages': [
    {'role': 'user', 'content': entry['question1']},
    {'role': 'assistant', 'content': entry['answer1']}
]})
guanaco_test = guanaco_test.map(lambda entry: {'messages': [
    {'role': 'user', 'content': entry['question1']},
    {'role': 'assistant', 'content': entry['answer1']}
]})

model.config.use_cache = False  # KV cache can only speed up inference, but we are doing training.

# Add low-rank adapters (LORA) to the model:
peft_config = LoraConfig(
    task_type='CAUSAL_LM',
    r=16,
    lora_alpha=32,  # thumb rule: lora_alpha should be 2*r
    lora_dropout=0.05,
    bias='none',
    target_modules='all-linear',
)
model = get_peft_model(model, peft_config)

training_arguments = SFTConfig(
    output_dir='output/phi-3.5-mini-instruct-guanaco-ddp-liger',
    per_device_train_batch_size=32//num_processes,  # Adjust per-device batch size for DDP
    gradient_accumulation_steps=1,
    gradient_checkpointing=True, # Gradient checkpointing improves memory efficiency, but slows down training,
        # e.g. Mistral 7B with PEFT using bitsandbytes:
        # - enabled: 11 GB GPU RAM and 8 samples/second
        # - disabled: 40 GB GPU RAM and 12 samples/second
    gradient_checkpointing_kwargs={'use_reentrant': False},  # Use newer implementation that will become the default.
    ddp_find_unused_parameters=False,  # Set to False when using gradient checkpointing to suppress warning message.
    log_level_replica='error',  # Disable warnings in all but the first process.
    optim='adamw_torch',
    learning_rate=2e-4,  # QLoRA suggestions: 2e-4 for 7B or 13B, 1e-4 for 33B or 65B
    logging_strategy='no',
    # logging_strategy='steps',  # 'no', 'epoch' or 'steps'
    # logging_steps=10,
    save_strategy='no',  # 'no', 'epoch' or 'steps'
    # save_steps=2000,
    # num_train_epochs=5,
    max_steps=20,
    bf16=True,  # mixed precision training
    report_to='none',  # disable wandb
    max_seq_length=1024,
)

def formatting_func(entry):
    return tokenizer.apply_chat_template(entry['messages'], tokenize=False)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=guanaco_train,
    eval_dataset=guanaco_test,
    processing_class=tokenizer,
    formatting_func=formatting_func,
)

if process_index == 0:  # Only print in first process.
    if hasattr(trainer.model, "print_trainable_parameters"):
        trainer.model.print_trainable_parameters()

# eval_result = trainer.evaluate()
# if process_index == 0:
#     print("Evaluation on test dataset before finetuning:")
#     print(eval_result)

train_result = trainer.train()
if process_index == 0:
    print("Training result:")
    print(train_result)

# eval_result = trainer.evaluate()
# if process_index == 0:
#     print("Evaluation on test dataset after finetuning:")
#     print(eval_result)

# Print memory usage once per node:
if local_process_index == 0:
    print_gpu_utilization()

# # Save model in first process only:
# if process_index == 0:
#     trainer.save_model()

Overwriting phi3_guanaco_ddp_liger.py


#### Next, we write the SLURM script and submit the script to the scheduler again:

In [7]:
%%writefile run_phi3_guanaco_ddp_liger.slurm
#!/bin/bash

#SBATCH --partition=boost_usr_prod
# #SBATCH --qos=boost_qos_dbg
#SBATCH --account=EUHPC_D20_063
#SBATCH --reservation=s_tra_ncc

## Specify resources:
## Leonardo Booster: 32 CPU cores and 4 GPUs per node => request 8 * number of GPUs CPU cores
## Leonardo Booster: 512 GB in total => request approx. 120 GB * number of GPUs requested
#SBATCH --nodes=1
#SBATCH --gpus-per-task=2  # up to 4 on Leonardo
#SBATCH --ntasks-per-node=1  # always 1
#SBATCH --mem=240GB  # should be 120GB * gpus-per-task on Leonardo
#SBATCH --cpus-per-task=16  # should be 8 * gpus-per-task on Leonardo

#SBATCH --time=0:30:00

# Load conda:
module purge
module load anaconda3
eval "$(conda shell.bash hook)"
conda activate /leonardo/pub/userexternal/mpfister/conda_env_martin24

# Include commands in output:
set -x

# Print current time and date:
date

# Print host name:
hostname

# List available GPUs:
nvidia-smi

# Set environment variables for communication between nodes:
export MASTER_PORT=$(shuf -i 20000-30000 -n 1)  # Choose a random port
export MASTER_ADDR=$(scontrol show hostnames ${SLURM_JOB_NODELIST} | head -n 1)
export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK

# Set launcher and launcher arguments:
export LAUNCHER="torchrun \
    --nnodes=$SLURM_JOB_NUM_NODES \
    --nproc_per_node=$SLURM_GPUS_ON_NODE \
    --rdzv_id=$SLURM_JOB_ID \
    --rdzv_endpoint=$MASTER_ADDR:$MASTER_PORT \
    --rdzv_backend=c10d"
# Set training script that will be executed:
export PROGRAM="phi3_guanaco_ddp_liger.py"

# Run:
time srun bash -c "$LAUNCHER $PROGRAM"
time srun bash -c "$LAUNCHER $PROGRAM --enable-liger"

Overwriting run_phi3_guanaco_ddp_liger.slurm


In [8]:
!sbatch run_phi3_guanaco_ddp_liger.slurm

Submitted batch job 12958288


In [9]:
!squeue --me

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          12958288 boost_usr run_phi3 mpfister CF       0:02      1 lrdn1261
          12952669 boost_usr jupyterl mpfister  R    5:03:54      1 lrdn3456


In [12]:
!cat slurm-12958288.out

+ date
Mon Feb 24 21:13:54 CET 2025
+ hostname
lrdn1261.leonardo.local
+ nvidia-smi
Mon Feb 24 21:13:54 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM-64GB            On | 00000000:56:00.0 Off |                    0 |
| N/A   43C    P0               67W / 477W|      0MiB / 65536MiB |      0%      Default |
|                                         |                      |        

#### Finally, we can clean up and delete the files that we just created:

In [6]:
!rm phi3_guanaco_ddp_liger.py run_phi3_guanaco_ddp_liger.slurm slurm-*.out